# Advanced Databases Project Phase 1
Academic Year: 2024/2025 <br>
Authors: Diogo Barreta 64560, Muhammed Jaabir Mohamed Zifri 64912, Josè Dalla Torre 64672

## Setup
First we need to install all the necessary packages.

In [ ]:
!pip install pandas
!pip install mysql-connector-python
!pip install matplotlib
!pip install pymongo
!pip install sqlalchemy

## Data visualization
Before starting to clean the data we need to load the csv as dataframe. <br>
After that we display the first rows to see the format of the data.

In [ ]:
import pandas as pd
import numpy as np
n_rows = 50000
folder_name = 'archive'
cards_df = pd.read_csv(f'{folder_name}/cards_data.csv',nrows=n_rows)
transactions_df = pd.read_csv(f'{folder_name}/transactions_data.csv',nrows=n_rows)
users_df = pd.read_csv(f'{folder_name}/users_data.csv',nrows=n_rows)

In [ ]:
cards_df.head()

In [ ]:
transactions_df.head()

In [ ]:
users_df.head()

### Missing Data
After the first exploration of the data we want to see if there are any missing values. <br>
To have a better understanding of the distribution of the values we use a countplot for every column of the dataframe.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import math

def create_plot(df: pd.DataFrame, df_name:str)-> None:
    n_cols = 3
    n_rows = math.ceil(len(df.columns) / n_cols)
    plt.tight_layout()
    plt.figure(figsize=(15, n_rows * 3))
    for i,column in enumerate(df.columns,1):
        # Count missing and valid values
        total = len(df[column])
        missing_count = df[column].isna().sum()
        valid_count = df[column].notna().sum()

        # Prepare data for plotting
        plot_data = pd.DataFrame({
            'Category': ['Missing Values', 'Valid Values'],
            'Count': [missing_count, valid_count]
        })

        # Create the countplot
        plt.suptitle(f"Analysing missing values of {df_name}", fontsize=14)
        ax = plt.subplot(n_rows,n_cols,i)
        sns.barplot(data=plot_data, x='Category', y='Count',hue='Category', palette=['red', 'green'])
        plt.title(f"Count of Missing vs Valid Values of {column}")
        plt.ylabel('Count')
        plt.xlabel('Category')
        for c in ax.containers:
            ax.bar_label(c, fmt=lambda x: f'{(x/total)*100:0.1f}%')
    plt.tight_layout()
    plt.show()

create_plot(cards_df, "Cards Data")
create_plot(transactions_df, "Transaction Data")
create_plot(users_df, "Users Data")

## Data Cleaning
We can see that only the transactions data has missing values. <br>
First we remove from the users data the unnecessary string like the dollar sign, then we convert all the column to the same value based on data present.

In [ ]:
remove_currency = lambda x: float(x.replace("$","").strip())
users_df["per_capita_income"] = users_df["per_capita_income"].apply(remove_currency)
users_df["yearly_income"] = users_df["yearly_income"].apply(remove_currency)
users_df["total_debt"] = users_df["total_debt"].apply(remove_currency)
users_df["gender"] = users_df["gender"].astype("string")
users_df["address"] = users_df["address"].astype("string")
users_df.dtypes

We apply the same logic to the cards dataframe. <br>
We decided to convert from "YES" or "NO" to boolean value to simplify the insertion in the database later. <br>
Also we decided to convert the date to the Datetime of pandas <br>

In [ ]:
cards_df["credit_limit"] = cards_df["credit_limit"].apply(remove_currency)
cards_df["has_chip"] = cards_df["has_chip"].replace({"YES": True, "NO": False})
cards_df["card_on_dark_web"] = cards_df["card_on_dark_web"].replace({"Yes": True, "No": False})
cards_df["card_brand"] = cards_df["card_brand"].astype("string")
cards_df["card_type"] = cards_df["card_type"].astype("string")
cards_df["card_number"] = cards_df["card_number"].astype("string")
cards_df["expires"] = pd.to_datetime(cards_df["expires"])
cards_df["acct_open_date"] = pd.to_datetime(cards_df["acct_open_date"])
cards_df.dtypes

Now it's time to clean the dataframe that contains the data about the transactions. <br>
We decided to fill the missing data of the **merchant_state** column as "Unkown" to prevent missing values. <br>
We decided to fill the missing data of the **zip** column as "0" to prevent missing values. <br>
We decided to fill the missing data of the **errors** column as "No Errors" to prevent missing values. <br>

In [ ]:
transactions_df["amount"] = transactions_df["amount"].apply(remove_currency)
transactions_df["date"] = pd.to_datetime(transactions_df["date"])
transactions_df["use_chip"] = transactions_df["use_chip"].astype("string")
transactions_df["merchant_city"] = transactions_df["merchant_city"].astype("string")
transactions_df["merchant_state"] = transactions_df["merchant_state"].astype("string")
transactions_df["merchant_state"] = transactions_df["merchant_state"].fillna('Unknown')
transactions_df['zip'] = transactions_df["zip"].fillna('0')
transactions_df['zip'] = transactions_df["zip"].astype(np.float64)
transactions_df["errors"] = transactions_df["errors"].astype("string")
transactions_df["errors"] = transactions_df["errors"].fillna("No Errors")
transactions_df.dtypes

We have manually changed few rows to work with one of the operations on the cards dataset by randomly assinging the value to True for `card_on_dark_web` column.

In [ ]:
card_ids = transactions_df[transactions_df['amount'] > 1000].sample(10)['card_id'].unique()
cards_df.loc[cards_df['id'].isin(card_ids), 'card_on_dark_web'] = True
cards_df.head()

In [ ]:
cards_df[cards_df.card_on_dark_web == True].shape

## MySQL
This section is dedicated to the definition of the tables, insertion and queries of the MySQL database.

### Setup
Since the components of the group are coding on different machine we decided to use a file called ".env" to store the configurations for the databases. <br>
The code to load the configuration is reported below

In [ ]:
import dotenv
import os
import mysql.connector
## database initialization
dotenv.load_dotenv()
database_name = 'group25'

MYSQL_CONFIG = {
    'username': os.environ['mysql_username'],
    'password': os.environ['mysql_password'],
    'host'    : os.environ['mysql_hostname'],
    'database_name' : database_name
}
# Create Connection
connection = mysql.connector.connect(
  host=MYSQL_CONFIG["host"],
  user=MYSQL_CONFIG["username"],
  password=MYSQL_CONFIG["password"]
)
db_url = f"mysql+mysqlconnector://{MYSQL_CONFIG['username']}:{MYSQL_CONFIG['password']}@{MYSQL_CONFIG['host']}:3306/{database_name}"
db_url

### Insertion into MySQL
Before we delete the database and the tables to make sure that we can create them correctly

In [ ]:
cursor = connection.cursor()
cursor.execute(f"DROP DATABASE IF EXISTS {database_name}")
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
cursor.execute(f"USE {database_name}")


cursor.execute(f"DROP TABLE IF EXISTS users")
cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
              id INT PRIMARY KEY,
              current_age INT,
              retirement_age INT,
              birth_year INT,
              birth_month INT,
              gender VARCHAR(255),
              address VARCHAR(255),
              latitude DECIMAL,
              longitude DECIMAL,
              per_capita_income DECIMAL,
              yearly_income DECIMAL,
              total_debt DECIMAL,
              credit_score INT,
              num_credit_cards INT
);
""")

cursor.execute(f"DROP TABLE IF EXISTS cards")
cursor.execute("""
CREATE TABLE IF NOT EXISTS cards (
      id INT PRIMARY KEY,
      client_id INT,
      card_brand VARCHAR(255),
      card_type VARCHAR(255),
      card_number VARCHAR(255),
      expires DATETIME,
      cvv INT,
      has_chip BOOLEAN,
      num_cards_issued INT,
      credit_limit DECIMAL,
      acct_open_date DATETIME,
      year_pin_last_changed INT,
      card_on_dark_web BOOLEAN,
      FOREIGN KEY (client_id) REFERENCES users(id)
);
""")

cursor.execute(f"DROP TABLE IF EXISTS transactions")
cursor.execute("""
CREATE TABLE IF NOT EXISTS transactions (
    id INT PRIMARY KEY,
    date DATETIME,
    client_id INT,
    card_id INT,
    amount DECIMAL,
    use_chip VARCHAR(255),
    merchant_id INT,
    merchant_city VARCHAR(255),
    merchant_state VARCHAR(255),
    zip DECIMAL,
    mcc INT,
    errors VARCHAR(255),
    FOREIGN KEY (client_id) REFERENCES users(id),
    FOREIGN KEY (card_id) REFERENCES cards(id)
);
""")
 

connection.commit()
cursor.execute("SHOW TABLES")
for table in cursor:
    print(table)

### Populate the Database
After that we populate the database as seen in class. <br>
We keep track of the time to compare later with the mongodb

In [ ]:
from sqlalchemy import create_engine, text
import time
mysql_times = []
engine = create_engine(db_url)
start_time = time.time()
users_df.to_sql('users', con=engine, if_exists='append', index=False, method='multi')
cards_df.to_sql('cards', con=engine, if_exists='append', index=False, method='multi')
transactions_df.to_sql('transactions', con=engine, if_exists='append', index=False, method='multi')
end_time = time.time()
mysql_times.append(end_time - start_time)
print("Data inserted into MySQL")

### Queries
Now we define the queries with a short description to help the reader

In [ ]:
queries = []

In [ ]:
q1 = {
    "id": 1,
    "description":"Select users with a credit score greater than 750.",
    "query" : "SELECT id, current_age, yearly_income, credit_score FROM users WHERE credit_score > 750;"
    }
queries.append(q1)

In [ ]:
q2 ={
    "id": 2,
    "description" :"Select all transactions where errors occurred.",
    "query" : "SELECT id, date, card_id, amount, errors FROM transactions WHERE errors != 'No Errors';"
}
queries.append(q2)

In [ ]:
q3 ={
    "id": 3,
    "description": "Calculate the total transaction amount per state for female users.",
     "query" : """
     INSERT INTO transactions (client_id, card_id, amount, date)
SELECT 
    u.id AS client_id, 
    c.id AS card_id, 
    -- You can specify the amount and transaction details here
    1000.00 AS amount, 
    CURRENT_TIMESTAMP AS transaction_date
FROM 
    users u
JOIN 
    cards c ON u.id = c.client_id
WHERE 
    u.gender = 'female'
AND 
    u.id = (
        SELECT client_id
        FROM transactions
        GROUP BY client_id
        ORDER BY SUM(amount) DESC
        LIMIT 1
    )
                """
}
queries.append(q3)  

In [ ]:
q4 = {
    "description": "Decrease credit limit by 100 of all cards flagged as on_dark_web that were used in transactions exceeding $1,000.",
    "query" : """UPDATE cards C
               JOIN transactions T ON T.card_id = C.id
               SET C.credit_limit = C.credit_limit - 100
               WHERE C.card_on_dark_web = 1 AND T.amount > 1000;"""
}
queries.append(q4)

Now we esecute them and we track the times to evaluate the performance later.

In [ ]:
with engine.connect() as conn:
    for q in queries:
        start_time = time.time()
        if "Select users with a credit score greater than 750." in q["description"]:
            result = conn.execute(text(q["query"]))
            results =  result.fetchall()
            print(f"{q['description']}: {len(results)} results found in {end_time - start_time:.4f} seconds")
        elif "Select all transactions where errors occurred." in q["description"]:
            result = conn.execute(text(q["query"]))
            results =  result.fetchall()
            print(f"{q['description']}: {len(results)} results found in {end_time - start_time:.4f} seconds")
        elif "Find all cards flagged as on_dark_web that were used in transactions exceeding $1,000." in q["description"]:
            result = conn.execute(text(q["query"]))
            results =  result.fetchall()
            print(f"{q['description']}: {len(results)} results found in {end_time - start_time:.4f} seconds")
        else:
            result = conn.execute(text(q["query"]))
            results = cursor.rowcount
            print(f"{q['description']}: {results} rows updated in {end_time - start_time:.4f} seconds")
        end_time = time.time()
        mysql_times.append(end_time - start_time)

## MongoDB

As seen before we utilize the configuration file to load the data

In [ ]:
from pymongo import MongoClient, ASCENDING, DESCENDING 
MONGO_DB_CONFIG = {
    'username': os.environ['mongo_username'],
    'password': os.environ['mongo_password'],
    'hostname' : os.environ['mongo_hostname'],
    'appName' : os.environ['mongo_appName'],
    'port'    : os.environ['mongo_port']
}


Then we connect to the database and we drop the collections

In [ ]:
uri = f'mongodb://{MONGO_DB_CONFIG["hostname"]}:{MONGO_DB_CONFIG["port"]}'
client = MongoClient(uri)
db = client[database_name]
db['users'].drop()
db['cards'].drop()
db['transactions'].drop()
print("Collections dropped")

Now we define the collections with the validator, we use a similar structure to the MySQL schema

In [ ]:
#Users collection
db.create_collection("users", validator = {
  "$jsonSchema": {
    "bsonType": "object",
    "required": ["id", "current_age", "retirement_age", "birth_year", "birth_month", "gender", "address", "latitude", "longitude", "per_capita_income", "yearly_income", "total_debt", "credit_score", "num_credit_cards"],
    "properties": {
      "id": {
        "bsonType": "int",
        "description": "Must be an integer and is required."
      },
      "current_age": {
        "bsonType": "int",
        "description": "Must be an integer and is required."
      },
      "retirement_age": {
        "bsonType": "int",
        "description": "Must be an integer and is required."
      },
      "birth_year": {
        "bsonType": "int",
        "description": "Must be an integer representing the year of birth and is required."
      },
      "birth_month": {
        "bsonType": "int",
        "description": "Must be an integer representing the month of birth and is required."
      },
      "gender": {
        "bsonType": "string",
        "enum": ["Male", "Female", "Other"],
        "description": "Must be a string and one of the predefined values."
      },
      "address": {
        "bsonType": "string",
        "description": "Must be a string representing the address and is required."
      },
      "latitude": {
        "bsonType": "double",
        "description": "Must be a float representing the latitude coordinate and is required."
      },
      "longitude": {
        "bsonType": "double",
        "description": "Must be a float representing the longitude coordinate and is required."
      },
      "per_capita_income": {
        "bsonType": "double",
        "description": "Must be a float formatted as currency and is required."
      },
      "yearly_income": {
        "bsonType": "double",
        "description": "Must be a float formatted as currency and is required."
      },
      "total_debt": {
        "bsonType": "double",
        "description": "Must be a float formatted as currency and is required."
      },
      "credit_score": {
        "bsonType": "int",
        "description": "Must be an integer representing the credit score and is required."
      },
      "num_credit_cards": {
        "bsonType": "int",
        "description": "Must be an integer representing the number of credit cards and is required."
      }
    }
  }
}
)

In [ ]:
db.create_collection("cards", validator = {
  "$jsonSchema": {
    "bsonType": "object",
    "required": [
      "id",
      "client_id",
      "card_brand",
      "card_type",
      "card_number",
      "expires",
      "cvv",
      "has_chip",
      "num_cards_issued",
      "credit_limit",
      "acct_open_date",
      "year_pin_last_changed",
      "card_on_dark_web"
    ],
    "properties": {
      "id": {
        "bsonType": "int",
        "description": "must be an integer and is required"
      },
      "client_id": {
        "bsonType": "int",
        "description": "must be an integer and is required"
      },
      "card_brand": {
        "bsonType": "string",
        "description": "must be a string and is required"
      },
      "card_type": {
        "bsonType": "string",
        "description": "must be a string and is required"
      },
      "card_number": {
        "bsonType": "string",
        "description": "must be a string and is required"
      },
      "expires": {
        "bsonType": "date",
        "description": "must be a date"
      },
      "cvv": {
        "bsonType": "int",
        "description": "must be an integer and is required"
      },
      "has_chip": {
        "bsonType": "bool",
        "description": "must be a boolean and is required"
      },
      "num_cards_issued": {
        "bsonType": "int",
        "description": "must be an integer greater than 0"
      },
      "credit_limit": {
        "bsonType": "double",
        "description": "must be a float and is required"
      },
      "acct_open_date": {
        "bsonType": "date",
        "description": "must be a date"
      },
      "year_pin_last_changed": {
        "bsonType": "int",
        "description": "must be an integer and is required"
      },
      "card_on_dark_web": {
        "bsonType": "bool",
        "description": "must be a boolean and is required"
      }
    }
  }
}
)

In [ ]:
#Transactions collection
db.create_collection("transactions", validator = {
    "$jsonSchema": {
      "bsonType": "object",
      "required": ["id", "date", "client_id", "card_id", "amount", "merchant_city", "merchant_state","errors"],
      "properties": {
        "id": {
          "bsonType": "int",
          "description": "Must be an integer and is required."
        },
        "date": {
          "bsonType": "date",
          "description": "Must be a date and is required"
        },
        "client_id": {
          "bsonType": "int",
          "description": "Must be an integer and is required."
        },
        "card_id": {
          "bsonType": "int",
          "description": "Must be an integer and is required."
        },
        "amount": {
          "bsonType": "double",
          "description": "Must be a float representing a monetary value and is required."
        },
        "use_chip": {
          "bsonType": "string",
          "description": "Must be a string indicating the transaction type if provided."
        },
        "merchant_id": {
          "bsonType": "int",
          "description": "Must be an integer if provided."
        },
        "merchant_city": {
          "bsonType": "string",
          "description": "Must be a string representing a city name and is required."
        },
        "merchant_state": {
          "bsonType": "string",
          "description": "Must be a string representing a state code and is required."
        },
        "zip": {
          "bsonType": "double",
          "description": "Must be a float representing a ZIP code if provided."
        },
        "mcc": {
          "bsonType": "int",
          "description": "Must be an integer representing a Merchant Category Code if provided."
        },
        "errors": {
          "bsonType": "string",
          "description": "Must be a string and is required."
        }
      }
    }
  }
)

In [ ]:
users_collection = db['users']
cards_collection = db['cards']
transactions_collection = db['transactions']

db.drop_collection('state_spending_summary')
db.create_collection("state_spending_summary")

Now we insert into the mongodb and we keep the times, similar to the section of MySQL

In [ ]:
### inserting the documents to the mongo db 
mongo_times = []
def insert_to_mongo_db(df: pd.DataFrame, c) -> None:
    collection = c
    initial_count = collection.count_documents({})
    print(f"# documents: {initial_count}.") 
    try:
        data_dict = df.to_dict(orient="records")
        result = collection.insert_many(data_dict, ordered=False)
        valid_count = collection.count_documents({}) - initial_count
        print(f"{valid_count} documents inserted.")

    except Exception as e:
        attempted_count = len(data_dict)
        valid_count = collection.count_documents({}) - initial_count
        print(f"{valid_count} documents inserted")
        print(f"{attempted_count - valid_count} documents failed to insert")


    print(f"# documents: {collection.count_documents({})}") 

start_time = time.time()
insert_to_mongo_db(users_df,users_collection)
insert_to_mongo_db(cards_df,cards_collection)
insert_to_mongo_db(transactions_df,transactions_collection)
end_time = time.time()
mongo_times.append(end_time - start_time)

Then we define the queries and execute them

In [ ]:
mongo_q1 = {"credit_score": {"$gt": 750}}

In [ ]:
mongo_q2 = ({"errors": {"$ne" : "No Errors"}})

In [ ]:
mongo_q3 = [
    {"$match": {"gender": "Female"}},
    {"$lookup": {
        "from": "transactions",  
        "localField": "id",  
        "foreignField": "client_id",  
        "as": "user_transactions"  
    }},
    {"$unwind": "$user_transactions"},
    {"$group": {
        "_id": "$user_transactions.merchant_state",  
        "total_amount": {
            "$sum": {"$toDouble": "$user_transactions.amount"}  
        }
    }},
    {"$sort": {"total_amount": -1}},
    {"$limit": 100}
]

In [ ]:
mongo_q4 = [
    {
        "$match": {
            "card_on_dark_web": True  
        }
    },
    {
        "$lookup": {
            "from": "transactions",
            "localField": "id",
            "foreignField": "card_id",
            "as": "transactions"
        }
    },
    {
        "$unwind": "$transactions"
    },
    {
        "$match": {
            "card_on_dark_web": True,
            "transactions.amount": {"$gt": 1000}
        }
    },
    {
        "$set": {
            "credit_limit": {"$multiply": ["$credit_limit", 0.5]}
        }
    },
    {
        "$merge": {
            "into": "cards",
            "whenMatched": "merge",
            "whenNotMatched": "fail"
        }
    }
]

In [ ]:
for q in queries:
    start_time = time.time()
    
    if "Select users with a credit score greater than 750." in q["description"]:
        query = mongo_q1
        result = users_collection.find(query)

    elif "Select all transactions where errors occurred." in q["description"]:
        query = mongo_q2
        result = transactions_collection.find(query)
    elif "Calculate the total transaction amount per state for female users." in q["description"]:
        query = mongo_q3
        result = users_collection.aggregate(query)
    else:
        query = mongo_q4
        result = cards_collection.aggregate(query)
        result = cards_collection.aggregate(query[:-2])
        

    end_time = time.time()
 
    mongo_times.append(end_time - start_time)
    list_result = list(result) 
    print(f"{q['description']}: {len(list_result)} found in {end_time - start_time:.4f} seconds")
client.close()

## Comparison
Now we want to evaluate the performance of MongoDB vs MySQL <br>
First we define a dataframe with the two lists that contain the times then we plot the result highlighting the differences in the execution time. <br>

In [ ]:
# Combine data into a DataFrame
data = pd.DataFrame({
    'Index': range(len(mongo_times)),
    'MongoDB': mongo_times,
    'MySQL': mysql_times
})
# Melt the DataFrame to plot later
melted_data = pd.melt(data, id_vars=['Index'], var_name='Database', value_name='Time')

# Create the barplot
plt.figure(figsize=(10, 6))
sns.barplot(data=melted_data, x='Index', y='Time', hue='Database', palette='viridis')
plt.title('MongoDB vs MySQL Execution Times')
plt.ylabel('Time (seconds)')
plt.xlabel('Index')
plt.yscale('log')  # Use log scale for better visualization of the range
plt.legend(title='Database')
plt.tight_layout()
plt.show()

The first index is related to the population of the database. <br>
The others four are related to the queries. <br>
As we can see MongoDB has a better performance overall expect in third query that has the worst time. It needs 28 second to execute the query while MySQL takes less than half of a second.

## Phase 2


### MySQL

In [ ]:
mysql_index_times = []

In [ ]:
cursor = connection.cursor()
cursor.execute("CREATE INDEX gender_index ON users (gender);")
cursor.execute("CREATE INDEX merchant_state_index ON transactions (merchant_state);")
print("\nIndex on 'gender' and 'merchant_state' created.")
query = q3["query"]
# Query with the indexes
print("\nQuerying with index (gender = 'Female' and merchant_state):")
start_time = time.time()
cursor.execute(query)
results = cursor.fetchall()
index_time = time.time() - start_time
mysql_index_times.append(index_time)
print(f"Number of results: {len(results)}")
print(f"Query time with index: {index_time:.4f} seconds")
cursor.execute(f"EXPLAIN ANALYZE {query}")
explain_result_after = cursor.fetchall()
print("Explain output with index:")
for row in explain_result_after:
    print(row)


sqlShowIndexes = "show index from users"
cursor.execute(sqlShowIndexes)
indexList = cursor.fetchall()
print(indexList)
sqlShowIndexes = "show index from transactions"
cursor.execute(sqlShowIndexes)
indexList = cursor.fetchall()
print(indexList)

cursor.execute("DROP INDEX gender_index ON users;")
cursor.execute("DROP INDEX merchant_state_index ON transactions;")

In [ ]:
cursor.execute("CREATE INDEX card_on_dark_web_index ON cards (card_on_dark_web);")
cursor.execute("CREATE INDEX amount_index ON transactions (amount);")
print("\Index on 'card_on_dark_web' and 'amount' created.")
query = q4["query"]
# Query with the index
print("\nQuerying with index (card_on_dark_web = '1' and amount >1000):")
start_time = time.time()
cursor.execute(query)
results = cursor.fetchall()
index_time = time.time() - start_time
mysql_index_times.append(index_time)
print(f"Number of results: {len(results)}")
print(f"Query time with index: {index_time:.4f} seconds")
cursor.execute(f"EXPLAIN ANALYZE {query}")
explain_result_after = cursor.fetchall()
print("Explain output with index:")
for row in explain_result_after:
    print(row)


sqlShowIndexes = "show index from cards"
cursor.execute(sqlShowIndexes)
indexList = cursor.fetchall()
print(indexList)
sqlShowIndexes = "show index from transactions"
cursor.execute(sqlShowIndexes)
indexList = cursor.fetchall()
print(indexList)

cursor.execute("DROP INDEX card_on_dark_web_index ON cards;")
cursor.execute("DROP INDEX amount_index ON transactions;")

cursor.close()
connection.close()

### MongoDB

In [ ]:
mongodb_index_times = []

In [ ]:
def indexes_information(collection):
    indexes_info = collection.index_information()
    print(indexes_info)
indexes_information(users_collection)
indexes_information(cards_collection)
indexes_information(transactions_collection)

In [ ]:
import pprint
# Create an index on 'gender'
users_collection.create_index([("gender", ASCENDING)])
print("\nIndex on 'gender' created.")
print(users_collection.index_information())
# Create an index on 'merchant_state'
transactions_collection.create_index([("merchant_state", ASCENDING)])
print("\nIndex on 'merchant_state' created.")
print(transactions_collection.index_information())
query = mongo_q3
# Query with the index
print("\nQuerying with index:")
results = list(users_collection.aggregate(query))
print(f"Number of results: {len(results)}")
print("\nTime in millis:")
explain_output = db.command(
    'explain',
    {
    'aggregate': 'users',
    'pipeline': query,
    'cursor': {}
    },
    verbosity = 'executionStats'
    )
pprint.pprint(explain_output["stages"][0]["$cursor"]["executionStats"]["executionTimeMillis"])
time_s = explain_output["stages"][0]["$cursor"]["executionStats"]["executionTimeMillis"]
mongodb_index_times.append(time_s/1000)
# Drop index
users_collection.drop_index('gender_1')
transactions_collection.drop_index('merchant_state_1')

In [ ]:
# Create an index on 'card_on_dark_web'
cards_collection.create_index([("card_on_dark_web", ASCENDING)])
print("\nIndex on 'card_on_dark_web' created.")
print(cards_collection.index_information())
# Create an index on 'amount'
transactions_collection.create_index([("amount", ASCENDING)])
print("\nIndex on 'amount' created.")
print(transactions_collection.index_information())
query = mongo_q4
# Query with the index
print("\nQuerying with index:")
results = list(cards_collection.aggregate(query))
print(f"Number of results: {len(results)}")
print("\nTime in millis:")
explain_output = db.command(
    'explain',
    {
    'aggregate': 'users',
    'pipeline': query,
    'cursor': {}
    },
    verbosity = 'executionStats'
    )
pprint.pprint(explain_output["stages"][0]["$cursor"]["executionStats"]["executionTimeMillis"])
time_s = explain_output["stages"][0]["$cursor"]["executionStats"]["executionTimeMillis"]
mongodb_index_times.append(time_s/1000)
# Drop index
cards_collection.drop_index('card_on_dark_web_1')
transactions_collection.drop_index('amount_1')

## Comparison

In [ ]:
data = pd.DataFrame({
    'Index': range(len(mysql_index_times)),
    'MongoDB': mongo_times[-2:],
    'MongoDB after indexing': mongodb_index_times,
    'MySQL': mysql_times[-2:],
    'MySQL after indexing': mysql_index_times
})
# Melt the DataFrame to plot later
melted_data = pd.melt(data, id_vars=['Index'], var_name='Database', value_name='Time')
# Create the barplot
plt.figure(figsize=(10, 6))
sns.barplot(data=melted_data, x='Index', y='Time', hue='Database', palette='viridis')
plt.title('MongoDB vs MySQL Execution Times')
plt.ylabel('Time (seconds)')
plt.xlabel('Index')
plt.yscale('log')  # Use log scale for better visualization of the range
plt.legend(title='Database')
plt.tight_layout()
plt.show()

## Optimization

In [ ]:
connection = mysql.connector.connect(
  host=MYSQL_CONFIG["host"],
  user=MYSQL_CONFIG["username"],
  password=MYSQL_CONFIG["password"]
)
cursor = connection.cursor()
cursor.execute(f"USE {database_name}")

In [ ]:
optimization_time = []

In [ ]:
cursor.execute(f"DROP TABLE IF EXISTS state_spending_summary")
cursor.execute('''CREATE TABLE IF NOT EXISTS state_spending_summary (
    state VARCHAR(255),
    total_spent DECIMAL );'''
)

st_time = time.time()
cursor.execute('''INSERT INTO state_spending_summary (state, total_spent)
SELECT T.merchant_state, SUM(T.amount) AS total_spent 
FROM transactions T
JOIN users U ON T.client_id = U.id
WHERE U.gender = 'Female'
GROUP BY T.merchant_state;
''')
end_time = time.time()
connection.commit()
optimization_time.append(end_time - st_time)

In [ ]:
st_time = time.time()
pipeline = [
    {
        "$lookup": {
            "from": "users",
            "localField": "client_id",
            "foreignField": "id",
            "as": "user_details"
        }
    },
    {
        "$unwind": "$user_details"
    },
    {
        "$match": {
            "user_details.gender": "Female"
        }
    },
    {
        "$group": {
            "_id": "$merchant_state",
            "total_spent": {"$sum": "$amount"}
        }
    },
    {
        "$project": {
            "state": "$_id",
            "total_spent": 1,
            "_id": 0
        }
    },
    {
        "$merge": {
            "into": "state_spending_summary",
            "whenMatched": "merge",
            "whenNotMatched": "insert"
        }
    }
]

transactions_collection.aggregate(pipeline)

end_time = time.time()

print(end_time - st_time)

In [ ]:
res = db.state_spending_summary.find({})
for r in res:
    print(r)

In [ ]:
pipeline = [
    {
        "$lookup": {
            "from": "transactions",
            "localField": "id",
            "foreignField": "card_id",
            "as": "transactions"
        }
    },
    {
        "$unwind": "$transactions"
    },
    {
        "$match": {
            "card_on_dark_web": True,
            "transactions.amount": {"$gt": 1000}
        }
    },
    {
        "$set": {
            "credit_limit": {"$multiply": ["$credit_limit", 0.5]}
        }
    },
    {
        "$merge": {
            "into": "cards",
            "whenMatched": "merge",
            "whenNotMatched": "fail"
        }
    }
]

cards_collection.aggregate(pipeline)

mongo db optimization problem version 2 

In [ ]:
q3opt = [
    {"$match": {"gender": "Female"}},  
    
    {"$lookup": {
        "from": "transactions",
        "localField": "id",
        "foreignField": "client_id",
        "as": "user_transactions"
    }},
    
    {"$match": {"user_transactions": {"$ne": []}}},  
    
    {"$unwind": "$user_transactions"},  
    
    {"$group": {
        "_id": "$user_transactions.merchant_state",
        "total_amount": {"$sum": {"$toDouble": "$user_transactions.amount"}}
    }},
    
    {"$sort": {"total_amount": -1}},
    
    {"$limit": 100}
]


users_collection.aggregate(q3opt)

In [ ]:
data